<a href="https://colab.research.google.com/github/anirbanghoshsbi/.github.io/blob/master/work/err/update_nifty.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import duckdb
import pandas as pd
import datetime

# Connect to the DuckDB database
db_path = "nifty50_data.duckdb"
conn = duckdb.connect(db_path)

# Step 1: Check the latest date in the database
query = "SELECT MAX(date) as latest_date FROM nifty50_table;"
latest_date = conn.execute(query).fetchone()[0]

# Convert to datetime for comparison
latest_date = pd.to_datetime(latest_date)

# Step 2: Get today's date and last trading day
today = datetime.date.today()

# If today is a weekend or holiday, adjust to last working day (basic adjustment)
if today.weekday() == 6:  # Sunday
    last_trading_day = today - datetime.timedelta(days=2)
elif today.weekday() == 5:  # Saturday
    last_trading_day = today - datetime.timedelta(days=1)
else:
    last_trading_day = today  # Assume today is a trading day (you may need to check actual market holidays)

# Step 3: Check if data is up to date
if latest_date.date() >= last_trading_day:
    print(f"Database is up to date. Latest date in DB: {latest_date.date()}")
else:
    print(f"Database is outdated. Latest date in DB: {latest_date.date()}, Expected: {last_trading_day}")

    # Prompt user for manual update
    new_data = input("Do you want to manually enter today's data? (yes/no): ").strip().lower()
    if new_data == "yes":
        # Collect manual data
        new_date = last_trading_day
        open_price = float(input("Enter Open Price: "))
        high_price = float(input("Enter High Price: "))
        low_price = float(input("Enter Low Price: "))
        close_price = float(input("Enter Close Price: "))

        # Insert the new data into the database
        insert_query = f"""
        INSERT INTO nifty50_data (date, open, high, low, close)
        VALUES ('{new_date}', {open_price}, {high_price}, {low_price}, {close_price});
        """
        conn.execute(insert_query)
        print(f"Data for {new_date} inserted successfully!")

# Close connection
conn.close()


Database is up to date. Latest date in DB: 2025-02-21


In [ ]:
# Connect to DuckDB
conn = duckdb.connect(db_path)

# Date to delete (wrong data)
wrong_date = "YYYY-MM-DD"  # Replace with the incorrect date

# Delete the incorrect entry
delete_query = f"DELETE FROM nifty50_table WHERE date = '{wrong_date}';"
conn.execute(delete_query)
print(f"Deleted incorrect data for {wrong_date}.")

# Ask for correct values
new_open = float(input("Enter correct Open Price: "))
new_high = float(input("Enter correct High Price: "))
new_low = float(input("Enter correct Low Price: "))
new_close = float(input("Enter correct Close Price: "))

# Reinsert the correct data
insert_query = f"""
INSERT INTO nifty50_table (date, open, high, low, close)
VALUES ('{wrong_date}', {new_open}, {new_high}, {new_low}, {new_close});
"""
conn.execute(insert_query)
print(f"Inserted correct data for {wrong_date}.")

# Close connection
conn.close()
